In [ ]:
!pip install html2text

In [ ]:
!pip install tika

In [ ]:
import html2text
import re
from datetime import datetime, timedelta
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join
from tika import parser


KoNLPY 설치

In [ ]:
!sudo apt-get install g++ openjdk-7-jdk

In [ ]:
!sudo apt-get install python-dev; pip install konlpy

In [ ]:
 !sudo apt-get install python3-dev; pip3 install konlpy   

mecab 설치

In [ ]:
!sudo apt-get install curl

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


eKoNLPY 설치

In [ ]:
!git clone https://github.com/entelecheia/eKoNLPy.git

In [ ]:
cd eKoNLPy

In [ ]:
!pip install .

In [ ]:
!pip install . --upgrade

ekonlpy 적용 함수 (순서대로 실행하세요~ 마지막 함수가 메인함수 입니다!)

In [ ]:
def pdf2txt (source_folder, output_folder) :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    for pdf in pdf_files :
        pdf_filepath = source_folder + pdf
        # pdf 파일을 text로 변환홓
        print(pdf_filepath)
        parsedPDF = parser.from_file(pdf_filepath)["content"]
        
        with open((output_folder + pdf).replace('.pdf', '.txt'), 'w') as f :
            f.write(parsedPDF)
            f.close()

In [ ]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [ ]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [ ]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
                
    return tokens + ngrams

메인함수

In [ ]:
# pdf 파일들이 있는 폴더 경로를 파라미터로 입력 받음(경로 끝에 반드시 '/'넣어야함!)
def bok_ekonlpy(source_folder):
  output_folder = source_folder + 'text2/'
  pdf2txt (source_folder, output_folder)

  txt_files = [f for f in listdir(output_folder) if isfile(join(output_folder, f))]
  txt_files.sort()

  for txt_file in txt_files :
        with open(output_folder + txt_file, 'r') as f :
            txt = f.read()
            sections, section_texts = preprocess_minutes(txt)
            print(txt_file)
            
            ngrams = []

            for section_text in section_texts[1:3] :
                for sentence in section_text :
                    ngrams.append(','.join(text2ngram(sentence)))
            return(ngrams)
   
#pdf 파일경로를 파라미터로 입력 받음(경로 끝에 반드시 '/'넣어야함!)
result = bok_ekonlpy('/content/drive/My Drive/bok/')
result

2020-07-31 08:25:10,035 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.


/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제23차)(2017.12.7.).pdf


2020-07-31 08:25:11,100 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-07-31 08:25:11,539 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제22차)(2017.11.30.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제21차)(2017.11.9.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제19차)(2017.10.19.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제18차)(2017.9.21.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제16차)(2017.8.31.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제15차)(2017.8.10.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제13차)(2017.7.13.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제12차)(2017.6.22.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제10차)(2017.5.25.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제8차)(2017.4.27.).pdf
/content/drive/My Drive/bok/금융통화위원회 의사록(2017년도 제7차)(2017.4.13.).pdf
/content/drive/My Dri

['경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없/VA,불구/NNG,원엔/NNG,환율/NNG,지속/NNG,로/NNG,하락/NNG,하/XSV,이유/NNG,물/VV,대해/VV,위안/NNG,절/NNG,상시/NNG,권/NNG,통화/NNG,절상/NNG,크/VA,기대/NNG,원화/NNG,달러/NNG,엔달러/NNG,환율/NNG,상승/NNG,추수/NNG,하/XSV,못하/VX,외환/NNG,수급/NNG,면/NNG,공급우위/NNG,상태/NNG,지속/NNG,되/XSV,국제투자은행/NNG,엔달러/NNG,환율/NNG,전망/NNG,상향/NNG,하/XSV,불구/NNG,원달러/NNG,조정/NNG,아직/MAG,이루/VV,않/VX,기업/NNG,기대/NNG,원화/NNG,절상/NNG,편향/NNG,되/XSV,기인/NNG,하/VV,설명/NNG,하/XSV,엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/NNG;절상/NNG,전망/NNG;상향/NNG',
 '환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반/NNG,하/VV,움직이/VV,점/NNG,감안/NNG,하/VV,때/NNG,원엔/NNG,환율/NNG,하락/NNG,일시적/VAX,현상/NNG,장기/NNG,반전/NNG,되/VV,가능성/NNG,있/VA,보인다는/VV,견해/NNG,밝혔으며/VV,대해/VV,견해/NNG,동의/NNG,하/XSV,시장/NNG,규모/NNG,작/VA,일시적/VAX,외부충격/NNG,의하/VV,쉽/VA,받/VV,점/NNG,설명/NNG,하/XSV,내외/NNG,금리/NNG,역전/NNG,나타나/VV,같/VA,대규모/NNG,외환/NNG,유입/NNG,일어나/VV,않/VX,감안/NNG,하/VV,때/NNG,원엔/NNG,환율/NNG,추가/NNG,하락/NNG,하/VV,점차/MAG,축소/NNG,되/VV,생각/NNG,되/VV,답변/NNG,하/XSV,원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일시적/VAX',
 '유로존/NNG,경